# Data

Inspect, clean and balance synthetic grasp samples.

In [2]:
import os
os.chdir('..')

In [3]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from vgn.io import *
from vgn.perception import *
from vgn.utils.transform import Rotation, Transform

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
import rospy
from vgn import vis

rospy.init_node("vgn_vis", anonymous=True)

[ERROR] [1684063775.696894]: Unable to immediately register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Path to the data folder.

In [47]:
root = Path("vgn/data/raw/foo")

## Inspection

Compute the number of positive and negative samples in the dataset.

In [48]:
df = read_df(root)

In [64]:
positives = df[df["label"] == 1]
negatives = df[df["label"] == 0]

print("Number of samples:", len(df.index))
print("Number of positives:", len(positives.index))
print("Number of negatives:", len(negatives.index))

Number of samples: 402
Number of positives: 201
Number of negatives: 201


Visualize a random sample. Make sure to have a ROS core running and open `config/sim.rviz` in RViz.

In [68]:
print(os.getcwd())
size, intrinsic, _, finger_depth = read_setup(root)
print(size)


/home/son22/robot_ws/src
0.30000000000000004


In [69]:
i = np.random.randint(len(df.index))
scene_id, grasp, label = read_grasp(df, i)
depth_imgs, extrinsics = read_sensor_data(root, scene_id)

tsdf = create_tsdf(size, 120, depth_imgs, intrinsic, extrinsics)
tsdf_grid = tsdf.get_grid()
cloud = tsdf.get_cloud()

vis.clear()
vis.draw_workspace(size)
vis.draw_points(np.asarray(cloud.points))
vis.draw_grasp(grasp, label, finger_depth)

IndexError: index 40 is out of bounds for axis 2 with size 40

Plot the distribution of angles between the gravity vector and $Z$ axis of grasps.

In [ ]:
angles = np.empty(len(positives.index))
for i, index in enumerate(positives.index):
    approach = Rotation.from_quat(df.loc[index, "qx":"qw"].to_numpy()).as_matrix()[:,2]
    angle = np.arccos(np.dot(approach, np.r_[0.0, 0.0, -1.0]))
    angles[i] = np.rad2deg(angle)        

In [ ]:
plt.hist(angles, bins=30)
plt.xlabel("Angle [deg]")
plt.ylabel("Count")
plt.show()

## Cleanup

DANGER: the following lines will modify/delete data.

Remove grasp positions that lie outside the workspace.

In [59]:
df = read_df(root)
df.drop(df[df["x"] < 0.02].index, inplace=True)
df.drop(df[df["y"] < 0.02].index, inplace=True)
df.drop(df[df["z"] < 0.02].index, inplace=True)
df.drop(df[df["x"] > 0.28].index, inplace=True)
df.drop(df[df["y"] > 0.28].index, inplace=True)
df.drop(df[df["z"] > 0.28].index, inplace=True)
write_df(df, root)

Remove unreferenced scenes.

In [66]:
df = read_df(root)
scenes = df["scene_id"].values
for f in (root / "scenes").iterdir():
    if f.suffix == ".npz" and f.stem not in scenes:
        print("Removed", f)
        f.unlink()

## Balance

Discard a subset of negative samples to balance classes.

In [61]:
df = read_df(root)

In [62]:
positives = df[df["label"] == 1]
negatives = df[df["label"] == 0]
i = np.random.choice(negatives.index, len(negatives.index) - len(positives.index), replace=False)
df = df.drop(i)

In [63]:
write_df(df, root)